In [104]:
import pandas as pd 
import re
from glob import glob
import random
import numpy as np
import copy

In [103]:
sapro_full = pd.read_excel('sapro_full.xlsx')

In [105]:
index_c = copy.copy(list(sapro_full.index))
np.random.shuffle(index_c)

In [106]:
sapro_shuffelled = sapro_full.loc[index_c].reset_index(drop=True).copy()

In [107]:
chunk_size = len(sapro_shuffelled) / 20

In [108]:
chunks = range(0, len(sapro_shuffelled), chunk_size)[:-1] + [8365]
chunks = zip(chunks[:-1], chunks[1:])

In [176]:
class_data = []
for start, stop in chunks:
    class_data.append(sapro_shuffelled.iloc[start:stop])

In [171]:
rank_mapping = {
    "C-1":"O-1",
    "C-2":"O-2",
    "C-3":"O-3",
    "C-4":"O-4",
    "M-1":"W-1",
    "M-2":"W-2",
    "M-3":"W-3"
}

In [172]:
sapro_shuffelled['Subject Pay Grade'] = sapro_shuffelled['Subject Pay Grade'].apply(lambda x: rank_mapping[x] if x in rank_mapping else x)

In [173]:
ordering = ["Cadet", "E-1", "E-2", "E-3", "E-4", "E-5", "E-6", 
 "E-7", "E-8", "E-9", "W-1", "W-2", "W-3", "W-4", "W-5", 
 "O-1", "O-2", "O-3", "O-4", "O-5", "O-6", "O-7", "O-8",
 "O-9", "O-10",]
# ordering ={k:i for i,k in enumerate( )}

In [174]:
sapro_shuffelled['Subject Pay Grade'] = (
    pd.Categorical(
        sapro_shuffelled['Subject Pay Grade'],
        ordering
    )
)

In [175]:
sapro_shuffelled['Outcome'] = pd.Categorical(sapro_shuffelled['Outcome'], ['no court-martial', 'not guilty', 'guilty'])

In [179]:
# Order by Acquittal (so people see "Acquittals" first)
# Order by Rank (so people read the convictions of higher-ranks first)
# Order by word count (so the lengthier cases are first ... those are more likely to be meaty)

def sort_df(df):
    df = df.iloc[(df['Case Synopsis Note']
          .fillna('')
          .apply(len)
          .pipe(lambda df: (-df).argsort())
         )].copy()
    df = df.sort_values('Subject Pay Grade', ascending=False).copy()
    df = df.sort_values('Outcome', ascending=True).copy()
    return df


In [189]:
class_data[0].shape

(418, 42)

In [184]:
! mkdir class_data

In [187]:
for i, df in enumerate(class_data):
    df = sort_df(df).copy()
    df.to_excel('class_data/sapro_data_set_%s.xlsx' % str(i) )